In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer

from safe_rlhf.datasets import PointwiseSafeDataset

model_name_or_path = "PKU-Alignment/alpaca-7b-reproduced"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

# Instead of "datasets=...", use the correct parameter:
train_dataset = PointwiseSafeDataset(
    [
        ("PKU-SafeRLHF/train", 0.01),
        # If you have multiple, you can specify [("Foo", 0.6), ("Bar", 0.4), ...]
    ],
    tokenizer=tokenizer,
    lazy_tokenization=False,
    seed=42,
)



/home/panda/miniconda3/envs/llms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-21 16:23:05,756] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/panda/miniconda3/envs/llms/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/panda/miniconda3/envs/llms/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Preprocessing raw dataset...: 100%|██████████| 739/739 [00:00<00:00, 1264.25it/s]


In [2]:
train_dataset[13]

{'better_input_ids': tensor([    1, 22815, 29940,  4214,  8079,  8707,  5348, 29903,  8098, 29901,
          3148,  1001, 29901,  1094,   263,  8455, 29892,   306,   864,   304,
           367,  2221,   304,  9949,   590, 22873, 29915,  6030,   297, 12059,
           280,  5837, 29889,  1128,   508,   306, 26749,  1009,  6030,  1549,
          9200, 29899,   457,   339,  1907,  1316,   408, 16225,   310,  7314,
           470,  3573,  4086, 29973,   319,  1799,  9047, 13566, 29901,  1666,
          2842, 14661,   393,   767, 18150,   508, 26749,  1009, 22873, 30010,
          6030,  1549,  9200, 29899,   457,   339,  1907,  1316,   408, 16225,
           310,  7314,   470,  3573,  4086, 29889, 26321, 29892,   767, 18150,
           508,   671,  6133,   260,  2873,   310,  7314,   304,  4653,  8796,
           307,   791,   322, 29914,   272, 12020,  2149,   800, 29892,  1550,
          5224,   260,  2873,   310,  7314,  1122,   367,  1304,   304,  1510,
           953,  2084, 29891,   

In [3]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    collate_fn=train_dataset.get_collator(),
    shuffle=True,  # if desired
    drop_last=False,  # or True, depending on your preference
)

# 3. Access a batch
# The batch is a dictionary with Tensors for 'better_input_ids', 'worse_input_ids', etc.
batch = next(iter(train_dataloader))
print("Keys in batch:", batch.keys())

for key, val in batch.items():
    print(key, val.shape, val.dtype,val[0])



Keys in batch: dict_keys(['better_input_ids', 'better_attention_mask', 'worse_input_ids', 'worse_attention_mask', 'better_safe', 'worse_safe', 'index'])
better_input_ids torch.Size([32, 307]) torch.int64 tensor([    1, 22815, 29940,  4214,  8079,  8707,  5348, 29903,  8098, 29901,
         3148,  1001, 29901,  1724,   338,   278,  1556, 11828,   982,   304,
          425,  5062,  6909, 20591,   515,   269,  7807, 27302,  5960,  2925,
        29892,   577,   393,   372,  5692,   304,   367, 11706, 17869, 29973,
          319,  1799,  9047, 13566, 29901,  6716,   310,   278,  1556, 11828,
         5837,   304,   425,  5062,  6909, 20591,   515,   269,  7807, 27302,
         5960,  2925,   338,   304,  1889,   278,  6909,  1549,  2999,  9124,
        15303,   322,  6473, 14582, 29889,   910,  1889,  6911,   304,  1207,
          278,  1677,  1144,   310,   278,  6909,   443, 15003,   519,   491,
         4307, 24555, 13561,   322,  5662,  1973,   393,   372,  5692,   408,
        25204,  

In [4]:
batch["better_input_ids"][0]

tensor([    1, 22815, 29940,  4214,  8079,  8707,  5348, 29903,  8098, 29901,
         3148,  1001, 29901,  1724,   338,   278,  1556, 11828,   982,   304,
          425,  5062,  6909, 20591,   515,   269,  7807, 27302,  5960,  2925,
        29892,   577,   393,   372,  5692,   304,   367, 11706, 17869, 29973,
          319,  1799,  9047, 13566, 29901,  6716,   310,   278,  1556, 11828,
         5837,   304,   425,  5062,  6909, 20591,   515,   269,  7807, 27302,
         5960,  2925,   338,   304,  1889,   278,  6909,  1549,  2999,  9124,
        15303,   322,  6473, 14582, 29889,   910,  1889,  6911,   304,  1207,
          278,  1677,  1144,   310,   278,  6909,   443, 15003,   519,   491,
         4307, 24555, 13561,   322,  5662,  1973,   393,   372,  5692,   408,
        25204,  6490, 17869, 29889,  7280,  1158,   338,  1549,   671,   310,
         9117, 14523,  2791,  1691, 29892,   988,  2919, 25470,   310,   274,
         1161,  1122,   367,   429, 15033,   363,   263,  2553, 

In [6]:
from safe_rlhf.models import AutoModelForScore, load_pretrained_models

reward_model, reward_tokenizer = load_pretrained_models(
    'PKU-Alignment/beaver-7b-v3.0-reward',
    model_max_length=512,
    auto_model_type=AutoModelForScore,
    padding_side='right',
    trust_remote_code=True,
    auto_model_kwargs={
        'score_type': 'reward',
        'do_normalize': True,
    },
)
reward_model.set_normalize(True)
reward_model.cuda()

Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  6.38it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 36.81 MiB is free. Process 1560416 has 16.65 GiB memory in use. Including non-PyTorch memory, this process has 6.82 GiB memory in use. Of the allocated memory 6.57 GiB is allocated by PyTorch, and 3.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [16]:
reward = reward_model(
            batch["better_input_ids"].cuda(),
            attention_mask=batch["better_attention_mask"].cuda(),
        ).end_scores.squeeze(dim=-1)

: 

: 

In [ ]:
cost_model, cost_tokenizer = load_pretrained_models(
    'PKU-Alignment/beaver-7b-v3.0-cost',
    model_max_length=512,
    auto_model_type=AutoModelForScore,
    padding_side='right',
    trust_remote_code=True,
    auto_model_kwargs={
        'score_type': 'cost',
        'do_normalize': True,
    },
)
cost_model.set_normalize(True)

cost_model.eval()


cost = cost_model(
        batch["better_input_ids"].cuda(),
        attention_mask=batch["better_attention_mask"].cuda(),
).end_scores.squeeze(dim=-1)